In [3]:
import torch 
import torchvision
import torch.nn as nn

import numpy as np
import math
import torchvision.transforms as transforms
from dataset_wrapper import get_pet_datasets
import matplotlib.pyplot as plt

In [4]:
batch_size = 4
img_h = 128;

transform = transforms.Compose([transforms.ToTensor()])

train_dataset, val_dataset, test_dataset = get_pet_datasets(img_width=img_h, img_height=img_h,root_path='./data' )

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


#selecting device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#printing because my main kernel wants to be stuck on CPU-only pytorch fsr
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)
# Mini-batch images and labels.
images_onebatch, labels = next(data_iter)
print(images_onebatch.shape)

GPU name: None
torch.Size([4, 3, 128, 128])


In [6]:
class AttentionBlock(nn.Module):

    def __init__(self, embed_dim, hidden_dim, num_heads, dropout=0.0):
        """
        Inputs:
            embed_dim - Dimensionality of input and attention feature vectors
            hidden_dim - Dimensionality of hidden layer in feed-forward network
                         (usually 2-4x larger than embed_dim)
            num_heads - Number of heads to use in the Multi-Head Attention block
            dropout - Amount of dropout to apply in the feed-forward network
        """
        super().__init__()

        self.layer_norm_1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads,
                                          dropout=dropout)
        self.layer_norm_2 = nn.LayerNorm(embed_dim)
        self.linear = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim), #expansion to four folder
            nn.GELU(), # Gaussian Error Linear Units (GELUs)
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, embed_dim), #reduce the dimensionality back
            nn.Dropout(dropout)
        )


    def forward(self, x):
        inp_x = self.layer_norm_1(x)
        x = x + self.attn(inp_x, inp_x, inp_x)[0]
        x = x + self.linear(self.layer_norm_2(x))
        return x

In [27]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module): # This class inherits from nn.Module and defines the architecture of the neural network.
    #In PyTorch, nn.Module is the base class for all neural network modules. It provides a set of functionalities that are 
    # commonly required when building neural networks, such as automatic gradient computation, model saving/loading, and simple layer management.
    def __init__(self, num_classes=4):
        # In PyTorch, custom neural networks are created by subclassing the nn.Module class and defining layers inside its __init__() method. 
        #The layers you define as class members (e.g., self.conv1, self.fc1) are automatically registered and tracked by PyTorch.
        super(ConvNet, self).__init__()

        #super() is used to call the __init__ method of the parent class nn.Module, without super(), you would lose the setup required for PyTorch
        #to properly manage the layers, parameters, and gradients in your neural network model.
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        self.sqex  = squeezeAndExcite(n_features)
        
    def forward(self, x):
        #The forward() method defines how the input data flows through the network (i.e., the computation graph). 
        #It's where you define the sequence of operations (e.g., layers, activations) the input goes through to produce the output.
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.sqex(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out) because we want to use this in squeeze and excitation
        return out

In [9]:
# Plot training loss
def generate_charts(num_layers, activation, batch_norm, residuals, train_losses, val_accuracies):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label=f'Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Training Loss Curve \n({num_layers} layers, {activation}, batchnorm={batch_norm}, residuals={residuals})')
    plt.legend()
    
    # Plot validation accuracy
    plt.subplot(1, 2, 2)
    plt.plot(val_accuracies, label=f'Validation Accuracy', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title(f'Validation Accuracy Curve \n({num_layers} layers, {activation}, batchnorm={batch_norm}, residuals={residuals})')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(f'layers{num_layers}_batch{batch_norm}_residuals{residuals}_{activation}.png')
    plt.show()

In [10]:
def test_model(model, descript, batch_size, device):
    model.eval()

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)
    # Disable gradient calculation for efficiency
    with torch.no_grad():
        correct = 0
        total = 0
        
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
    
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            
        new_acc_descript = '       Test Accuracy of the model on the 10000 test images: {:.2f} %'.format(100 * correct / total)
        append_text_to_file('AllAccuracies.txt', descript + "\n" + new_acc_descript)
        print(descript + "\n" + new_acc_descript)

In [11]:
def append_text_to_file(file_path, text_to_append):
    try:
        with open(file_path, 'a') as file:
            file.write(text_to_append + '\n')
        print(f"Text appended to {file_path} successfully.")
    except Exception as e:
        print(f"Error: {e}")

In [12]:
import re
def extract_models_from_file(filepath):
    models_ive_trained = []

    # Read file
    with open(filepath, "r") as file:
        lines = file.readlines()

    # Define a regex pattern to capture required parameters
    pattern = re.compile(
        r"pos_embedding: (\w+); num_heads: (\d+); num_layers:(\d+); patch_size: (\d+)",
        re.IGNORECASE
    )

    # Process every line to match pattern
    for line in lines:
        match = pattern.search(line)
        if match:
            pos_embedding, num_heads, num_layers, patch_size = match.groups()
            model_string = f"{pos_embedding}{num_heads}{num_layers}{patch_size}"
            models_ive_trained.append(model_string)

    return models_ive_trained

In [13]:
def test_model(model, descript, batch_size, device):
    # --- Test the model ---
    model.eval()  # evaluation mode
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)# choose the class that have the highest score
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
        print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
        #Remember guessing randomly among 10 classes would be about 25% accuracy
    
    new_acc_descript = f'       Test Accuracy of the model on the {total} test images: {(100 * correct / total)}'
    append_text_to_file('Part2AllAccuracies.txt', descript + "\n" + new_acc_descript)
    print(descript + "\n" + new_acc_descript)

In [31]:
def validate_model(model):
    model.eval()
    correct = 0                       
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy
    #print(f'Validation Accuracy after epoch {epoch+1}: {accuracy:.2f}%')

In [19]:
class squeezeAndExcite(nn.Module):
    #The block has a convolutional block as an input... called by ConvNet with 16 features
    def __init__(self, n_features, reduction=16):
        print(ConvNet.shape)

    def forward(self, x):
        out = F.avg_pool2d(x, kernel_size=x.size()[2:4])

In [28]:
def run_model():
    #hyperparameters
    num_epochs = 25
    num_classes = 4
    batch_size = 100
    learning_rate = 0.001

    #selecting device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #printing because my main kernel wants to be stuck on CPU-only pytorch fsr

    model = ConvNet(num_classes=num_classes).to(device)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    train_losses = []
    val_accuracies = []
    
    # Train the model
    total_step = len(train_loader)

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            # Forward pass
            outputs = model(images)
            loss = criterion(newOut, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()                            
            if (i+1) % 50 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        
        avg_loss = epoch_loss / total_step
        train_losses.append(avg_loss)

        #now we do validation. exit training mode
        val_accuracies.append(validate_model(model))
        
        

run_model()

torch.Size([4, 32, 32, 32])


TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not squeezeAndExcite